In [1]:
import numpy as np

# load the file
data_k = np.load("/k_final.npy")
data_s = np.load("/s_final.npy")
data_p = np.load("/p_final.npy")

target_k = np.array([0] * data_k.shape[0])
target_s = np.array([1] * data_s.shape[0])
target_p = np.array([2] * data_p.shape[0])

print(type(data_k), type(data_s), type(data_p), "\n")
print(data_k.shape, data_s.shape, data_p.shape, "\n")
print(data_k[:5],"\n\n", data_s[:5],"\n\n", data_p[:5])

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> 

(100, 50176) (100, 50176) (100, 50176) 

[[ 0  0  0 ...  0  0  0]
 [61 55 60 ... 13 13 13]
 [61 51 49 ... 23 22 22]
 [ 0  0  1 ...  2  2  1]
 [ 0  0  0 ...  0  0  0]] 

 [[13 14 17 ...  5  8 14]
 [59 60 61 ... 33 32 31]
 [16 16 18 ... 31 31 17]
 [45 42 44 ... 54 62 68]
 [38 38 38 ... 43 43 43]] 

 [[  0   0   0 ...   0   0   0]
 [ 88  88  88 ... 102 102 102]
 [ 96  96  96 ...  96  96  96]
 [  1   1   1 ...   0   0   0]
 [ 95  95  95 ...  95  95  95]]


In [2]:
data_all = np.concatenate([data_k, data_s, data_p], axis=0)
target_all = np.concatenate([target_k, target_s, target_p], axis=0)

print("Final shape:", data_all.shape, target_all.shape)
print(data_all[:5])

Final shape: (300, 50176) (300,)
[[ 0  0  0 ...  0  0  0]
 [61 55 60 ... 13 13 13]
 [61 51 49 ... 23 22 22]
 [ 0  0  1 ...  2  2  1]
 [ 0  0  0 ...  0  0  0]]


In [3]:
from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(data_all, target_all,
                                                     test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_val.shape, y_val.shape)

(192, 50176) (192,)
(60, 50176) (60,)
(48, 50176) (48,)


In [4]:
import numpy as np

# Add Gaussian noise
def add_noise(data, noise_factor=0.05):
    noise = np.random.normal(0, noise_factor, data.shape)
    return data + noise

# Random scaling (multiply by random factor per feature)
def random_scaling(data, scale_range=(0.9, 1.1)):
    scale = np.random.uniform(scale_range[0], scale_range[1], size=(1, data.shape[1]))
    return data * scale

# Random shift (add small value per feature)
def random_shift(data, shift_range=(-0.1, 0.1)):
    shift = np.random.uniform(shift_range[0], shift_range[1], size=(1, data.shape[1]))
    return data + shift

# Feature dropout (randomly set some features to 0)
def feature_dropout(data, drop_prob=0.1):
    mask = np.random.binomial(1, 1 - drop_prob, size=data.shape)
    return data * mask

def feature_shuffle(data, shuffle_prob=0.1):
    data_aug = data.copy()
    n_features = data.shape[1]
    for f in range(n_features):
        if np.random.rand() < shuffle_prob:
            np.random.shuffle(data_aug[:, f])
    return data_aug

def gaussian_blur(data, sigma=1):
    from scipy.ndimage import gaussian_filter1d
    return gaussian_filter1d(data, sigma=sigma, axis=0)

def random_feature_swap(data, swap_prob=0.05):
    data_aug = data.copy()
    n_features = data.shape[1]
    for i in range(data.shape[0]):
        if np.random.rand() < swap_prob:
            f1, f2 = np.random.choice(n_features, 2, replace=False)
            data_aug[i, f1], data_aug[i, f2] = data_aug[i, f2], data_aug[i, f1]
    return data_aug

def random_sign_flip(data, flip_prob=0.05):
    mask = np.random.binomial(1, flip_prob, size=data.shape)
    return data * ((-1) ** mask)

def mixup(data, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    indices = np.random.permutation(data.shape[0])
    return lam * data + (1 - lam) * data[indices]

def random_erasing(data, erase_prob=0.05, erase_size=2):
    data_aug = data.copy()
    n_features = data.shape[1]
    for i in range(data.shape[0]):
        if np.random.rand() < erase_prob:
            start = np.random.randint(0, n_features - erase_size + 1)
            data_aug[i, start:start+erase_size] = 0
    return data_aug

def random_projection(data, n_components=None):
    if n_components is None:
        n_components = data.shape[1]
    random_matrix = np.random.normal(size=(data.shape[1], n_components))
    return np.dot(data, random_matrix)

In [5]:
# concatenate along rows (axis=0)
data_augmented_train = np.concatenate(
    [X_train, add_noise(X_train, noise_factor=0.1), random_scaling(X_train), random_shift(X_train), feature_dropout(X_train, drop_prob=0.2),
     feature_shuffle(X_train), gaussian_blur(X_train), random_feature_swap(X_train), random_sign_flip(X_train), mixup(X_train),
     random_erasing(X_train)],
    axis=0
)
target_augmented_train = np.concatenate(
    [y_train, y_train, y_train, y_train, y_train, y_train, y_train, y_train, y_train, y_train, y_train],
    axis=0
)

print("Original shape:", X_train.shape, y_train.shape)
print("Augmented shape:", data_augmented_train.shape, target_augmented_train.shape)

Original shape: (192, 50176) (192,)
Augmented shape: (2112, 50176) (2112,)


In [6]:
X_train = data_augmented_train.reshape(-1, 224, 224, 1)
y_train = target_augmented_train
X_val   = X_val.reshape(-1, 224, 224, 1)
X_test  = X_test.reshape(-1, 224, 224, 1)

In [7]:
import tensorflow as tf

In [8]:
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

X_train_rgb = tf.image.grayscale_to_rgb(tf.convert_to_tensor(X_train))
X_val_rgb   = tf.image.grayscale_to_rgb(tf.convert_to_tensor(X_val))

print("Train shape:", X_train_rgb.shape)
print("Val shape:", X_val_rgb.shape)

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(X_train_rgb)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    "/content/drive/MyDrive/checkpoints/epoch_{epoch:02d}_valacc_{val_accuracy:.2f}.keras",
    save_weights_only=False,
    save_freq="epoch"
)

from tensorflow.keras.applications import EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

optimizer_a = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(
    optimizer=optimizer_a,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Callback
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

# Train
history = model.fit(
    datagen.flow(X_train_rgb, y_train, batch_size=32),
    validation_data=(X_val_rgb, y_val),
    epochs=200,
    batch_size=32,
    callbacks=[early_stop, checkpoint]
)

Train shape: (2112, 224, 224, 3)
Val shape: (48, 224, 224, 3)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 133s 764ms/step - accuracy: 0.3661 - loss: 1.1949 - val_accuracy: 0.3333 - val_loss: 1.2161
Epoch 2/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 28s 418ms/step - accuracy: 0.4135 - loss: 1.1021 - val_accuracy: 0.4375 - val_loss: 1.0945
Epoch 3/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 40s 408ms/step - accuracy: 0.4335 - loss: 1.0601 - val_accuracy: 0.4167 - val_loss: 1.0390
Epoch 4/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 27s 410ms/step - accuracy: 0.4753 - loss: 1.0372 - val_accuracy: 0.4375 - val_loss: 1.0108
Epoch 5/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 41s 409ms/step - accuracy: 0.4830 - loss: 1.0093 - val_accuracy: 0.5208 - val_loss: 0.9950
Epoch 6/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 41s 411ms/step - accuracy: 0.5372 - loss: 0.9459 - val_accuracy: 0.5000 - val_loss: 0.9773
Epoch 7/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 27s 414ms/step - accuracy: 0.5647 - loss: 0.9343 - val_accuracy: 0.5833 - val_loss: 0.9633
Epoch 8/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 41s 411ms/step - accuracy: 0.5768 - loss: 0.8956 -

In [9]:
# Make sure test data is numpy with correct dtype
X_test = np.array(X_test, dtype=np.float32)
X_test_rgb   = tf.image.grayscale_to_rgb(tf.convert_to_tensor(X_test))
y_test = np.array(y_test, dtype=np.int32)  # since you used sparse_categorical_crossentropy

# Evaluate
test_loss, test_acc = model.evaluate(X_test_rgb, y_test, batch_size=32)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

2/2 ━━━━━━━━━━━━━━━━━━━━ 18s 11s/step - accuracy: 0.7493 - loss: 0.6122
Test Loss: 0.6462701559066772
Test Accuracy: 0.7333333492279053


In [11]:
!ls /content/drive/MyDrive/checkpoints/

epoch_01_valacc_0.33.keras  epoch_28_valacc_0.79.keras
epoch_01_valacc_0.40.h5     epoch_29_valacc_0.79.keras
epoch_02_valacc_0.44.keras  epoch_30_valacc_0.79.keras
epoch_02_valacc_0.46.h5     epoch_31_valacc_0.81.keras
epoch_03_valacc_0.42.keras  epoch_32_valacc_0.81.keras
epoch_03_valacc_0.54.h5     epoch_33_valacc_0.81.keras
epoch_04_valacc_0.44.keras  epoch_34_valacc_0.83.keras
epoch_05_valacc_0.52.keras  epoch_35_valacc_0.83.keras
epoch_06_valacc_0.50.keras  epoch_36_valacc_0.81.keras
epoch_07_valacc_0.58.keras  epoch_37_valacc_0.83.keras
epoch_08_valacc_0.65.keras  epoch_38_valacc_0.85.keras
epoch_09_valacc_0.62.keras  epoch_39_valacc_0.85.keras
epoch_10_valacc_0.69.keras  epoch_40_valacc_0.83.keras
epoch_11_valacc_0.77.keras  epoch_41_valacc_0.85.keras
epoch_12_valacc_0.69.keras  epoch_42_valacc_0.85.keras
epoch_13_valacc_0.75.keras  epoch_43_valacc_0.88.keras
epoch_14_valacc_0.73.keras  epoch_44_valacc_0.85.keras
epoch_15_valacc_0.71.keras  epoch_45_valacc_0.85.keras
epoch_16_v